In [30]:
import os
import pandas as pd
import xgboost
import utils
import scoring
import swifter
from sklearn.model_selection import train_test_split
import catboost

In [2]:
# Test
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/dd6255a1-a14b-4276-9a2b-db7f360e01c7
# Train
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/3a5e940c-2382-4716-9ff7-8fbc269b98ac

In [3]:
train = utils.load_full_train_csv(path='.')
# test = pd.read_csv(os.path.join(DATA_PATH, "test-features.csv.gz"), index_col="id", usecols=utils.SIMPLE_FEATURE_COLUMNS + ["id"])

In [4]:
# closest_hits_features = train.swifter.apply(utils.find_closest_hit_per_station, result_type="expand", axis=1)
# To save the result of this long calculation (takes ~30min)
# closest_hits_features.to_pickle('train_closest_hit.pkl')
closest_hits_features = pd.read_pickle('train_closest_hit.pkl')

In [5]:
new_columns = ['s'+str(i) for i in range(24)]
closest_hits_features.columns = new_columns

In [6]:
train_concat = pd.concat([train, closest_hits_features],axis=1)

In [7]:
train_concat.columns

Index(['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]', 'avg_cs[1]',
       'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_

In [8]:
test.head()

NameError: name 'test' is not defined

In [9]:
train_part, validation = train_test_split(train_concat, test_size=0.25, shuffle=True, random_state=2342234)

In [36]:
model = xgboost.XGBClassifier(learning_rate=0.1)
# model = catboost.CatBoostClassifier()

In [37]:
model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS+new_columns].values,
          train_part.label.values,
          # cat_features=['ndof'],
          sample_weight=train_part.kinWeight.values,
      )

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [38]:
validation_predictions = model.predict_proba(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS+new_columns].values)[:, 1]

In [39]:
scoring.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)

0.6793380663465934

In [ ]:
model.fit(train.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.kinWeight.values)

In [ ]:
predictions = model.predict_proba(test.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [ ]:
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "submission.csv.gz", index_label=utils.ID_COLUMN)

In [16]:
for c in train_concat.columns:
    print(c, train_concat.head()[c])

ncl[0] id
0     79.0
1     33.0
2    115.0
3     28.0
4     50.0
Name: ncl[0], dtype: float32
ncl[1] id
0    18.0
1    14.0
2    53.0
3     8.0
4     8.0
Name: ncl[1], dtype: float32
ncl[2] id
0    22.0
1     6.0
2    23.0
3     5.0
4     4.0
Name: ncl[2], dtype: float32
ncl[3] id
0    10.0
1    15.0
2    25.0
3    15.0
4    13.0
Name: ncl[3], dtype: float32
avg_cs[0] id
0    2.392405
1    1.575758
2    2.843478
3    2.964286
4    2.400000
Name: avg_cs[0], dtype: float32
avg_cs[1] id
0    2.055556
1    2.285714
2    2.433962
3    1.500000
4    1.250000
Name: avg_cs[1], dtype: float32
avg_cs[2] id
0    1.590909
1    1.333333
2    1.826087
3    1.800000
4    1.500000
Name: avg_cs[2], dtype: float32
avg_cs[3] id
0    1.200000
1    1.400000
2    1.280000
3    1.000000
4    1.076923
Name: avg_cs[3], dtype: float32
ndof id
0    8.0
1    8.0
2    8.0
3    8.0
4    8.0
Name: ndof, dtype: float32
MatchedHit_TYPE[0] id
0    2.0
1    2.0
2    2.0
3    2.0
4    2.0
Name: MatchedHit_TYPE[0], dtype:

In [29]:
train_concat['Lextra_X[4]'].hist()

KeyError: 'Lextra_X[4]'